In [2]:
# -*- coding: utf-8 -*-
# 选项 1: 端到端多模态处理 RAG 实现示例
# 使用 CLIP 进行图文统一嵌入，使用 MLLM (占位符) 进行生成

import sqlite3
import os
import numpy as np
from typing import List, Dict, Union, Optional, Tuple
import faiss  # 确保已安装 faiss-cpu 或 faiss-gpu
from transformers import CLIPProcessor, CLIPModel # 确保已安装 transformers 和 pillow
from PIL import Image # 确保已安装 pillow
import torch # transformers 依赖 torch
import zhipuai # 导入 ZhipuAI 客户端 (用于占位符 Generator)
import json # 导入 json 库用于读取数据文件
import time # 导入 time 库用于增加延迟
import random # 导入 random 库用于随机选择图片

# --- 数据加载与图片关联函数 (与原示例相同) ---
def load_data_from_json_and_associate_images(json_path: str, image_dir: str) -> List[Dict]:
    """
    # 从 JSON 文件加载文档数据，并尝试在指定的图像目录中关联对应的图片文件。
    # (与原示例代码相同，此处省略详细注释)
    """
    if not os.path.exists(json_path):
        print(f"错误: 未找到 JSON 文件 '{json_path}'。")
        return []
    documents = []
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff']
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
    except Exception as e:
        print(f"错误: 读取或解析 JSON 文件 '{json_path}' 失败: {e}")
        return []
    for item in json_data:
        doc_id = item.get('name')
        text_content = item.get('description')
        if not doc_id or not text_content:
            continue
        image_path = None
        if image_dir and os.path.exists(image_dir):
             for ext in image_extensions:
                 potential_image_path = os.path.join(image_dir, str(doc_id) + ext)
                 if os.path.exists(potential_image_path):
                     image_path = potential_image_path
                     break
        documents.append({
            'id': str(doc_id),
            'text': str(text_content) if text_content is not None else None,
            'image_path': image_path
        })
    print(f"已加载 {len(json_data)} 条记录，成功准备 {len(documents)} 个文档。")
    return documents

# --- 1. 多模态编码器 (MultimodalEncoder - CLIP) ---
# 使用 CLIP 模型将文本和图像编码到 *同一个* 向量空间
class MultimodalEncoder:
    """
    # 使用 Hugging Face Transformers 的 CLIP 模型进行多模态编码。
    # 负责将文本和图像分别转换为向量，这些向量在同一个语义空间中可比较。
    """
    def __init__(self, model_name: str = "openai/clip-vit-base-patch32", device: Optional[str] = None):
        try:
            self.processor = CLIPProcessor.from_pretrained(model_name)
            self.model = CLIPModel.from_pretrained(model_name)
            self.vector_dimension = self.model.text_model.config.hidden_size

            if device:
                self.device = torch.device(device)
            elif torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                self.device = torch.device("cpu")

            self.model.to(self.device)
            self.model.eval()
            print(f"MultimodalEncoder (CLIP) 初始化成功，模型: {model_name}, 维度: {self.vector_dimension}, 设备: {self.device}")

        except Exception as e:
             print(f"加载 CLIP 模型 {model_name} 失败: {e}")
             raise e

    def encode_text(self, text: str) -> Optional[np.ndarray]:
        """编码单个文本字符串"""
        if not text: return None
        try:
            with torch.no_grad():
                inputs = self.processor(text=text, return_tensors="pt", padding=True, truncation=True).to(self.device)
                text_features = self.model.get_text_features(**inputs)
                vector = text_features.squeeze().cpu().numpy().astype('float32')
                norm = np.linalg.norm(vector)
                return vector / norm if norm > 1e-6 else np.zeros_like(vector)
        except Exception as e:
            print(f"编码文本 '{text[:30]}...' 出错: {e}")
            return None

    def encode_image(self, image_path: str) -> Optional[np.ndarray]:
        """编码单个图像文件"""
        if not image_path or not os.path.exists(image_path):
            if image_path: print(f"警告: 未找到图像文件 {image_path}。")
            return None
        try:
            with torch.no_grad():
                image = Image.open(image_path).convert("RGB")
                inputs = self.processor(images=image, return_tensors="pt").to(self.device)
                image_features = self.model.get_image_features(**inputs)
                vector = image_features.squeeze().cpu().numpy().astype('float32')
                norm = np.linalg.norm(vector)
                return vector / norm if norm > 1e-6 else np.zeros_like(vector)
        except Exception as e:
            print(f"编码图像 {image_path} 出错: {e}")
            return None

    def encode_query(self, text: Optional[str] = None, image_path: Optional[str] = None) -> Optional[np.ndarray]:
        """
        # 对查询进行编码。如果同时提供文本和图像，简单地平均它们的向量。
        # 注意：更高级的融合策略可能效果更好。
        """
        text_vec = self.encode_text(text) if text else None
        image_vec = self.encode_image(image_path) if image_path else None

        if text_vec is None and image_vec is None:
            print("错误: 查询必须至少包含文本或有效图像路径。")
            return None
        elif text_vec is not None and image_vec is not None:
            # 对文本和图像向量取平均作为融合表示
            combined_vector = np.mean([text_vec, image_vec], axis=0).astype('float32')
            norm = np.linalg.norm(combined_vector)
            return combined_vector / norm if norm > 1e-6 else np.zeros_like(combined_vector)
        elif text_vec is not None:
            return text_vec
        else: # image_vec is not None
            return image_vec


# --- 2. 索引器 (Indexer) - 选项 1 ---
# 编码文本和图像，将 *两种* 向量存入 Faiss，原始数据存入 SQLite
class Indexer_Option1:
    """
    # 索引器 (选项1):
    # - 使用 MultimodalEncoder (CLIP) 分别编码文本块和图像。
    # - 将文本向量和图像向量都存储到 Faiss 索引中。
    # - 将原始文本和图像路径存储到 SQLite 数据库中。
    # - 使用唯一的 vector_index_id 关联 Faiss 中的向量和 SQLite 中的原始数据。
    """
    def __init__(self, db_path: str, faiss_index_path: str, clip_model_name: str):
        self.db_path = db_path
        self.faiss_index_path = faiss_index_path
        self.encoder = MultimodalEncoder(clip_model_name)
        self.vector_dimension = self.encoder.vector_dimension

        self._init_db()
        self._load_or_create_faiss_index()

    def _init_db(self):
        """初始化 SQLite 数据库和表"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                # vector_index_id: Faiss 中的唯一 ID，也是 DB 主键
                # doc_id: 原始文档标识符 (来自 JSON 'name')
                # content_type: 'text' 或 'image'
                # text_content: 存储原始文本 (如果是文本类型)
                # image_path: 存储图像路径 (如果是图像类型)
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS multimodal_items (
                        vector_index_id INTEGER PRIMARY KEY,
                        doc_id TEXT NOT NULL,
                        content_type TEXT NOT NULL CHECK(content_type IN ('text', 'image')),
                        text_content TEXT,
                        image_path TEXT
                    )
                ''')
                # 添加索引以加速 doc_id 查找 (用于去重)
                cursor.execute("CREATE INDEX IF NOT EXISTS idx_doc_id_type ON multimodal_items (doc_id, content_type)")
                conn.commit()
                print(f"数据库已初始化或连接成功: {self.db_path}")
        except Exception as e:
             print(f"初始化数据库失败: {e}")
             raise e

    def _load_or_create_faiss_index(self):
        """加载或创建 Faiss 向量索引 (IndexIDMap2 + 内积)"""
        try:
            if os.path.exists(self.faiss_index_path):
                self.index = faiss.read_index(self.faiss_index_path)
                print(f"成功加载 Faiss 索引: {self.faiss_index_path}, 包含 {self.index.ntotal} 个向量。")
            else:
                print("未找到 Faiss 索引文件，创建新的空索引。")
                quantizer = faiss.IndexFlatIP(self.vector_dimension) # 内积 (余弦相似度)
                self.index = faiss.IndexIDMap2(quantizer)
                print(f"创建了新的 Faiss IndexIDMap2 (内积) 索引 (维度: {self.vector_dimension})。")
        except Exception as e:
            print(f"加载或创建 Faiss 索引失败: {e}。将创建一个新的空索引。")
            quantizer = faiss.IndexFlatIP(self.vector_dimension)
            self.index = faiss.IndexIDMap2(quantizer)

    def index_documents(self, documents: List[Dict]):
        """
        # 对文档列表进行索引。
        # 对每个文档，分别索引其文本内容和关联的图像（如果存在）。
        """
        if not documents: return
        print(f"开始处理 {len(documents)} 个文档进行多模态索引...")

        vectors_to_add = []
        vector_ids_for_batch = [] # Faiss 需要的 int64 ID
        data_to_store = [] # (vector_id, doc_id, content_type, text, image_path)

        start_vector_index_id = self.index.ntotal
        next_vector_index_id = start_vector_index_id

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        processed_count = 0
        skipped_duplicates = 0
        skipped_encoding_errors = 0

        for doc in documents:
            doc_id = doc.get('id')
            text = doc.get('text')
            image_path = doc.get('image_path')

            if not doc_id: continue

            # --- 索引文本部分 ---
            if text:
                # 检查文本是否已索引
                cursor.execute("SELECT 1 FROM multimodal_items WHERE doc_id = ? AND content_type = 'text'", (doc_id,))
                if cursor.fetchone():
                    skipped_duplicates += 1
                    # print(f"  文本部分 (ID: {doc_id}) 已存在，跳过。")
                    pass # 避免重复打印
                else:
                    text_vector = self.encoder.encode_text(text)
                    if text_vector is not None:
                        vectors_to_add.append(text_vector)
                        vector_ids_for_batch.append(next_vector_index_id)
                        data_to_store.append((next_vector_index_id, doc_id, 'text', text, None))
                        next_vector_index_id += 1
                    else:
                        skipped_encoding_errors += 1
                        # print(f"  文本部分 (ID: {doc_id}) 编码失败，跳过。")

            # --- 索引图像部分 ---
            if image_path and os.path.exists(image_path):
                 # 检查图像是否已索引
                 cursor.execute("SELECT 1 FROM multimodal_items WHERE doc_id = ? AND content_type = 'image'", (doc_id,))
                 if cursor.fetchone():
                     skipped_duplicates += 1
                     # print(f"  图像部分 (ID: {doc_id}) 已存在，跳过。")
                     pass
                 else:
                    image_vector = self.encoder.encode_image(image_path)
                    if image_vector is not None:
                        vectors_to_add.append(image_vector)
                        vector_ids_for_batch.append(next_vector_index_id)
                        data_to_store.append((next_vector_index_id, doc_id, 'image', None, image_path))
                        next_vector_index_id += 1
                    else:
                         skipped_encoding_errors += 1
                         # print(f"  图像部分 (ID: {doc_id}, Path: {image_path}) 编码失败，跳过。")
            elif image_path:
                 # print(f"  警告: 图像文件不存在，无法索引图像部分 (ID: {doc_id}, Path: {image_path})")
                 pass

            processed_count += 1
            if processed_count % 50 == 0: # 每处理50个文档打印一次进度
                 print(f"  已处理 {processed_count}/{len(documents)} 个原始文档...")


        # --- 批量添加到 Faiss 和 SQLite ---
        if vectors_to_add:
            try:
                vectors_np = np.array(vectors_to_add, dtype='float32')
                ids_np = np.array(vector_ids_for_batch, dtype='int64')
                self.index.add_with_ids(vectors_np, ids_np)
                print(f"成功向 Faiss 添加 {len(vectors_np)} 个新向量。当前总数: {self.index.ntotal}")

                cursor.executemany(
                    "INSERT INTO multimodal_items (vector_index_id, doc_id, content_type, text_content, image_path) VALUES (?, ?, ?, ?, ?)",
                    data_to_store
                )
                conn.commit()
                print(f"成功向数据库存储 {len(data_to_store)} 条新记录。")

            except Exception as e:
                print(f"批量添加向量或存储数据时出错: {e}")
                conn.rollback()
        else:
            print("没有新的有效向量或数据需要添加到索引或数据库。")

        conn.close()
        print(f"索引过程完成。总处理文档: {processed_count}, 跳过重复: {skipped_duplicates}, 跳过编码错误: {skipped_encoding_errors}")

    def get_item_by_vector_index_id(self, vector_index_id: int) -> Optional[Dict]:
        """根据 Faiss 返回的 vector_index_id 从数据库获取原始条目信息"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute(
                    "SELECT doc_id, content_type, text_content, image_path FROM multimodal_items WHERE vector_index_id = ?",
                    (vector_index_id,)
                )
                row = cursor.fetchone()
                if row:
                    doc_id, content_type, text_content, image_path = row
                    return {
                        'vector_index_id': vector_index_id,
                        'doc_id': doc_id,
                        'content_type': content_type,
                        'text': text_content,
                        'image_path': image_path
                    }
                return None
        except Exception as e:
             print(f"从数据库根据 vector_index_id {vector_index_id} 获取条目出错: {e}")
             return None

    def get_index_count(self) -> int:
         return self.index.ntotal if hasattr(self, 'index') else 0

    def get_db_item_count(self) -> int:
         try:
             with sqlite3.connect(self.db_path) as conn:
                 cursor = conn.cursor()
                 cursor.execute("SELECT COUNT(*) FROM multimodal_items")
                 return cursor.fetchone()[0]
         except Exception: return 0

    def save_index(self):
        if hasattr(self, 'index') and self.index.ntotal > 0:
            try:
                faiss.write_index(self.index, self.faiss_index_path)
                print(f"Faiss 索引已保存到: {self.faiss_index_path}")
            except Exception as e: print(f"保存 Faiss 索引失败: {e}")
        elif hasattr(self, 'index'): print("Faiss 索引为空，跳过保存。")
        else: print("Faiss 索引未初始化，跳过保存。")

    def close(self):
        self.save_index()

# --- 3. 检索器 (Retriever) - 选项 1 ---
# 使用与 Indexer 相同的 Encoder 对查询编码，在 Faiss 中搜索，获取原始数据
class Retriever_Option1:
    """
    # 检索器 (选项1):
    # - 使用 Indexer 的 MultimodalEncoder 对用户查询（文本、图像、图文）进行编码。
    # - 在 Faiss 索引（包含文本和图像向量）中搜索最相似的向量。
    # - 根据返回的 vector_index_id 从数据库获取对应的原始文本或图像信息。
    """
    def __init__(self, indexer: Indexer_Option1):
        if not isinstance(indexer, Indexer_Option1):
             raise ValueError("必须提供有效的 Indexer_Option1 实例。")
        self.indexer = indexer
        self.encoder = indexer.encoder
        self.index = indexer.index
        if self.index is None or self.index.ntotal == 0:
            print("警告: 检索器初始化时发现索引为空。")

    def retrieve(self, query: Union[str, Dict], k: int = 5) -> List[Dict]:
        """
        # 执行检索流程。

        Args:
            query: 用户查询 (str for text, dict for image {'image_path': ...}, or dict for multimodal {'text': ..., 'image_path': ...})。
            k: 检索数量。

        Returns:
            包含原始数据（文本或图像路径）和得分的字典列表。
            [{'doc_id': ..., 'content_type': ..., 'text': ..., 'image_path': ..., 'score': ...}, ...]
        """
        if self.index is None or self.index.ntotal == 0:
            print("错误: 向量索引不可用或为空。")
            return []

        # 1. 查询编码
        query_vector = None
        if isinstance(query, str):
            query_vector = self.encoder.encode_query(text=query)
        elif isinstance(query, dict):
            query_vector = self.encoder.encode_query(text=query.get('text'), image_path=query.get('image_path'))
        else:
            print(f"错误: 不支持的查询类型 {type(query)}")
            return []

        if query_vector is None:
            print("错误: 查询编码失败。")
            return []

        query_vector = query_vector.reshape(1, self.indexer.vector_dimension)

        # 2. Faiss 搜索
        try:
            print(f"  - 正在 Faiss 索引中搜索 Top {k}...")
            scores, vector_index_ids = self.index.search(query_vector, k)
            print(f"  - Faiss 搜索完成，找到 {len(vector_index_ids[0])} 个结果。")
        except Exception as e:
            print(f"  - Faiss 搜索失败: {e}")
            return []

        # 3. 获取原始数据
        retrieved_items = []
        print("  - 正在从数据库获取原始条目信息...")
        for i, vec_id in enumerate(vector_index_ids[0]):
            if vec_id == -1: continue # Faiss 可能返回 -1
            item_data = self.indexer.get_item_by_vector_index_id(int(vec_id))
            if item_data:
                item_data['score'] = float(scores[0][i]) # 添加相似度得分 (内积)
                retrieved_items.append(item_data)
            else:
                print(f"  - 警告: 未找到 vector_index_id {vec_id} 对应的数据库记录。")
        print(f"  - 成功获取 {len(retrieved_items)} 个条目的数据。")
        return retrieved_items

    def close(self): pass # 无需特殊操作

# --- 4. 生成器 (Generator) - 选项 1 (使用 MLLM - 占位符) ---
class Generator_Option1:
    """
    # 生成器 (选项1):
    # - **需要使用真正的多模态大语言模型 (MLLM)，如 GPT-4V, GLM-4V, LLaVA 等。**
    # - 接收检索到的原始文本和图像。
    # - 构建能够被 MLLM API 接受的 Prompt (包含文本和图像引用/数据)。
    # - 调用 MLLM API 生成答案。
    #
    # !!! 注意: 下面的实现使用 ZhipuAI 文本模型作为占位符 !!!
    # !!! 实际应用中需要替换为与所选 MLLM 匹配的 API 调用和 Prompt 构建逻辑 !!!
    """
    def __init__(self, api_key: Optional[str] = None, model_name: str = "glm-4v", # 默认指向多模态模型
                 fallback_text_model: str = "glm-4-flash"):
        # 优先使用传入的 api_key，否则从环境变量 ZHIPUAI_API_KEY 获取
        final_api_key = api_key if api_key else os.getenv("ZHIPUAI_API_KEY")
        if not final_api_key:
            raise ValueError("未提供 ZHIPUAI_API_KEY。")

        try:
            self.client = zhipuai.ZhipuAI(api_key=final_api_key)
            self.model_name = model_name # 目标 MLLM 模型
            self.fallback_model = fallback_text_model # 备用文本模型
            print(f"Generator (选项 1 - MLLM 占位符) 初始化成功。目标模型: {self.model_name}, 备用: {self.fallback_model}")
            # 在这里可以添加检查 API Key 是否支持目标 MLLM 的逻辑 (如果 Zhipu SDK 支持)
        except Exception as e:
             print(f"初始化智谱 AI 客户端失败: {e}")
             raise e

    def generate(self, query: Union[str, Dict], context: List[Dict]) -> str:
        """
        # 接收查询和包含原始文本/图像的上下文，生成答案。

        Args:
            query: 原始用户查询 (文本或字典)。
            context: 从 Retriever_Option1 获取的列表，每个元素包含原始数据。
                     [{'doc_id': ..., 'content_type': 'text'/'image', 'text': ..., 'image_path': ..., 'score': ...}, ...]

        Returns:
            MLLM (或占位符 LLM) 生成的文本响应。
        """
        print(f"正在使用 {len(context)} 个检索到的条目 (含图文) 为查询生成响应...")

        # --- !!! MLLM Prompt 构建逻辑 (需要根据实际 MLLM API 调整) !!! ---
        # 1. 提取查询中的文本部分
        query_text = query if isinstance(query, str) else query.get('text', "请描述您看到的内容。") # 如果查询是图片，提供默认文本

        # 2. 分离上下文中的文本和图像
        context_texts = []
        context_images = [] # 存储图像路径或 base64 编码等 MLLM 需要的信息
        for item in context:
            if item['content_type'] == 'text' and item['text']:
                # 限制文本长度
                truncated_text = item['text'][:500] + ('...' if len(item['text']) > 500 else '')
                context_texts.append(f"- [来源 Doc ID: {item['doc_id']}, 得分: {item['score']:.4f}] {truncated_text}")
            elif item['content_type'] == 'image' and item['image_path']:
                # MLLM 通常需要图像的 URL 或 Base64 编码
                # 这里仅作为示例，存储路径和元信息
                context_images.append({
                    "path": item['image_path'],
                    "doc_id": item['doc_id'],
                    "score": item['score']
                })
                # !! 实际操作: 需要将 image_path 转换为 MLLM API 接受的格式 !!
                # 例如: 读取图片 -> base64 编码，或者如果图片有公共 URL 则使用 URL

        # 3. 构建 MLLM 需要的 messages 结构 (示例，具体格式依赖于 MLLM API)
        #    - 对于 GLM-4V, content 可以是包含 text 和 image_url 的列表
        #    - 对于 GPT-4V, content 可以是包含 text 和 image_url (含 base64) 的列表
        #    - 对于 LLaVA, 可能需要特定的格式或库来处理

        messages = []
        system_prompt = f"""
        你是一个多模态助手，擅长结合文本和图像信息回答问题。请根据用户查询和下面提供的文本及图像信息进行回答。
        严格根据提供的信息作答，如果信息不足请说明。
        参考文本信息:
        {chr(10).join(context_texts) if context_texts else '无相关文本信息。'}
        """.strip()
        messages.append({"role": "system", "content": system_prompt})

        # 构建 user message (包含文本查询和图像引用)
        user_content_parts = [{"type": "text", "text": query_text}]
        if context_images:
            user_content_parts.append({"type": "text", "text": "\n请参考以下图像:"})
            image_count = 0
            for img_info in context_images:
                # --- !!! 此处需要将图像转换为 MLLM API 格式 !!! ---
                # 假设 MLLM API 支持 "image_url" 包含本地路径 (很多 API 不支持) 或 base64
                # 这是一个 *占位符* 结构，需要替换
                try:
                    # 尝试读取并 base64 编码 (如果需要)
                    # img_base64 = image_to_base64(img_info['path']) # 需要实现 image_to_base64
                    # user_content_parts.append({
                    #     "type": "image_url",
                    #     "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"} # 示例 base64 格式
                    # })
                    # 或者如果 API 支持直接引用，可能更简单，但通常不安全或不可行
                    # 为了代码能运行，暂时只添加文本描述
                    user_content_parts.append({
                        "type": "text",
                        "text": f"\n- 图像 {image_count+1} (来源 Doc ID: {img_info['doc_id']}, 路径: {img_info['path']}, 得分: {img_info['score']:.4f}) [图像内容需由 MLLM 查看]"
                    })
                    image_count += 1
                except Exception as img_err:
                    print(f"处理图像 {img_info['path']} 时出错: {img_err}")
                    user_content_parts.append({"type": "text", "text": f"\n- 图像 (来源 Doc ID: {img_info['doc_id']}) 加载失败."})


        messages.append({"role": "user", "content": user_content_parts}) # user content 是一个列表

        print("  - [占位符] MLLM Prompt 构建完成 (需要适配实际 MLLM API 格式)。")
        # print("  - [占位符] Prompt 预览 (结构):", messages) # 打印复杂结构

        # --- !!! 调用 MLLM API (占位符逻辑) !!! ---
        # 实际应调用目标 MLLM 的 API，如 self.client.chat.completions.create(model=self.model_name, messages=messages, ...)
        # 由于没有通用的本地 MLLM 调用方式或统一 API，这里使用 ZhipuAI 文本模型作为回退演示
        print(f"  - [占位符] 正在尝试调用目标 MLLM '{self.model_name}'...")
        try:
            # --- !!! 此处应为实际 MLLM API 调用 !!! ---
            # response = self.client.chat.completions.create(model=self.model_name, messages=messages, ...)

            # --- 回退到文本模型进行演示 ---
            print(f"  - [占位符] 无法直接调用 MLLM，将使用备用文本模型 '{self.fallback_model}' 处理纯文本部分...")
            text_only_messages = [
                {"role": "system", "content": system_prompt}, # 复用 system prompt
                {"role": "user", "content": query_text + "\n\n(注意: 我无法直接看到图片，请根据提供的文本描述回答。)"} # 提醒文本模型
            ]
            response = self.client.chat.completions.create(
                model=self.fallback_model, # 使用备用文本模型
                messages=text_only_messages,
                temperature=0.7,
                max_tokens=1024
            )
            llm_response = response.choices[0].message.content
            print(f"  - [占位符] 使用备用文本模型调用成功。")

        except Exception as e:
            print(f"  - 调用 MLLM (或备用模型) 时出错: {e}")
            return f"调用 LLM 出错: {e}"

        # 后处理
        processed_response = llm_response.strip()
        print("  - 响应生成和后处理完成。")
        return processed_response

# --- 选项 1 示例使用流程 ---
if __name__ == "__main__":
    print("\n" + "="*10 + " 选项 1: 端到端多模态 RAG 示例 " + "="*10 + "\n")

    # --- 配置 ---
    json_data_path = 'data.json'
    image_directory_path = 'images'
    db_file_opt1 = 'rag_option1.db'
    faiss_index_file_opt1 = 'rag_option1.faiss'
    CLIP_MODEL_OPT1 = "openai/clip-vit-base-patch32"
    # !! 选择你的目标 MLLM 模型名称 (如果可用), 否则用文本模型作为占位符 !!
    TARGET_MLLM = "glm-4v" # 假设的目标 MLLM
    FALLBACK_TEXT_LLM = "glm-4-flash" # 备用文本模型

    # --- 清理 ---
    print("--- 清理旧文件 ---")
    if os.path.exists(db_file_opt1): os.remove(db_file_opt1)
    if os.path.exists(faiss_index_file_opt1): os.remove(faiss_index_file_opt1)
    print("--- 清理完成 ---")

    # --- 1. 加载数据 ---
    print("\n--- 步骤 1: 加载数据 ---")
    documents = load_data_from_json_and_associate_images(json_data_path, image_directory_path)
    if not documents: exit("错误: 未加载到文档。")
    print("--- 步骤 1 完成 ---")

    # --- 2. 初始化 Indexer 并索引 ---
    print("\n--- 步骤 2: 初始化 Indexer (选项 1) 并索引 ---")
    indexer_opt1 = None
    try:
        indexer_opt1 = Indexer_Option1(db_path=db_file_opt1, faiss_index_path=faiss_index_file_opt1, clip_model_name=CLIP_MODEL_OPT1)
        indexer_opt1.index_documents(documents)
        print(f"索引完成。向量总数: {indexer_opt1.get_index_count()}, 数据库条目数: {indexer_opt1.get_db_item_count()}")
        if indexer_opt1.get_index_count() == 0:
             print("错误: 索引为空！")
             indexer_opt1 = None
    except Exception as e:
        print(f"Indexer 初始化或索引失败: {e}")
        indexer_opt1 = None
    print("--- 步骤 2 完成 ---")

    # --- 3. 初始化 Retriever ---
    print("\n--- 步骤 3: 初始化 Retriever (选项 1) ---")
    retriever_opt1 = None
    if indexer_opt1:
        try:
            retriever_opt1 = Retriever_Option1(indexer=indexer_opt1)
            print("Retriever 初始化成功。")
        except Exception as e:
            print(f"Retriever 初始化失败: {e}")
    else:
        print("Indexer 不可用，跳过 Retriever 初始化。")
    print("--- 步骤 3 完成 ---")

    # --- 4. 初始化 Generator ---
    print("\n--- 步骤 4: 初始化 Generator (选项 1 - MLLM 占位符) ---")
    generator_opt1 = None
    if os.getenv("ZHIPUAI_API_KEY"):
        try:
            generator_opt1 = Generator_Option1(model_name=TARGET_MLLM, fallback_text_model=FALLBACK_TEXT_LLM)
            print("Generator 初始化成功。")
        except Exception as e:
            print(f"Generator 初始化失败: {e}")
    else:
        print("ZHIPUAI_API_KEY 未设置，跳过 Generator 初始化。")
    print("--- 步骤 4 完成 ---")

    # --- 5. 执行查询 ---
    print("\n--- 步骤 5: 执行查询示例 ---")
    if retriever_opt1 and generator_opt1:
        # 辅助函数打印结果
        def print_retrieved_items_opt1(items: List[Dict]):
            if not items: print("    (未检索到条目)")
            for i, item in enumerate(items):
                score_str = f"得分: {item.get('score', 'N/A'):.4f}"
                print(f"    {i+1}. [类型: {item['content_type']}] ID: {item['doc_id']}, {score_str}")
                if item['content_type'] == 'text':
                    text_preview = item['text'][:100] + ('...' if len(item['text']) > 100 else '')
                    print(f"       文本: {text_preview}")
                elif item['content_type'] == 'image':
                    print(f"       图像文件: {os.path.basename(item['image_path'])}")
                print("    " + "-" * 15)

        # 查询示例
        queries_opt1 = [
            "带隙基准电路的核心原理是什么？", # 文本查询
            {'image_path': random.choice([d['image_path'] for d in documents if d['image_path']]) if any(d['image_path'] for d in documents) else None}, # 图像查询 (随机选一张)
            {'text': '请结合这张图片解释电路如何工作', 'image_path': random.choice([d['image_path'] for d in documents if d['image_path']]) if any(d['image_path'] for d in documents) else None} # 多模态查询
        ]
        # 过滤掉无效查询
        queries_opt1 = [q for q in queries_opt1 if q and (not isinstance(q, dict) or q.get('image_path'))]

        for i, query in enumerate(queries_opt1):
            print(f"\n--- 查询 {i+1}/{len(queries_opt1)} ---")
            if isinstance(query, str): print(f"查询 (文本): {query}")
            elif 'text' not in query: print(f"查询 (图像): {os.path.basename(query['image_path'])}")
            else: print(f"查询 (多模态): 文本='{query['text']}', 图像='{os.path.basename(query['image_path'])}'")

            try:
                retrieved_context = retriever_opt1.retrieve(query, k=5)
                print(f"  检索到 {len(retrieved_context)} 个条目:")
                print_retrieved_items_opt1(retrieved_context)

                if retrieved_context:
                    print("\n  生成响应 (使用 MLLM 或占位符):")
                    response = generator_opt1.generate(query, retrieved_context)
                    print(response)
                else:
                    print("\n  未检索到上下文，无法生成响应。")
            except Exception as e:
                print(f"\n  执行查询或生成时出错: {e}")
            time.sleep(1)

    else:
        print("Retriever 或 Generator 未初始化，跳过查询执行。")
    print("--- 步骤 5 完成 ---")

    # --- 6. 清理 ---
    print("\n--- 步骤 6: 清理资源 ---")
    if indexer_opt1: indexer_opt1.close()
    if retriever_opt1: retriever_opt1.close()
    # Generator 无需关闭
    print("--- 清理完成 ---")
    print("\n" + "="*10 + " 选项 1 示例结束 " + "="*10 + "\n")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



========== 选项 1: 端到端多模态 RAG 示例 ==========

--- 清理旧文件 ---
--- 清理完成 ---

--- 步骤 1: 加载数据 ---
已加载 219 条记录，成功准备 219 个文档。
--- 步骤 1 完成 ---

--- 步骤 2: 初始化 Indexer (选项 1) 并索引 ---


Error while downloading from https://cdn-lfs.hf.co/openai/clip-vit-base-patch32/a63082132ba4f97a80bea76823f544493bffa8082296d62d71581a4feff1576f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1745643762&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTY0Mzc2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9vcGVuYWkvY2xpcC12aXQtYmFzZS1wYXRjaDMyL2E2MzA4MjEzMmJhNGY5N2E4MGJlYTc2ODIzZjU0NDQ5M2JmZmE4MDgyMjk2ZDYyZDcxNTgxYTRmZWZmMTU3NmY%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=WtEzPrpaLcBeKtb5b%7EeJwccvR2UvoMT%7EOXfvLl-Eshtla5DwjXfhlCllW4eAgEQ3MUbT9N9yBlgnK5RuRN67v%7E2y1qKTD%7Ez1y748uLny6g3srLjMapjYGNqZMn66jSc9djX-y7LTTG2Szvvo1GZczLqWjaRTB%7E1Lf8ZeNFmI8cH2REy07yNP8gfnGIL4pmpdUktRgt2RRq8TU1lSkR7OIamy3YMLiXNFfWjaPpIhO06CMUuI8DwxD7Q8Dh3uLi4GlZG2JOQBmsga%7E3BYqVcVzKJcgr4jrc

MultimodalEncoder (CLIP) 初始化成功，模型: openai/clip-vit-base-patch32, 维度: 512, 设备: cpu
数据库已初始化或连接成功: rag_option1.db
未找到 Faiss 索引文件，创建新的空索引。
创建了新的 Faiss IndexIDMap2 (内积) 索引 (维度: 512)。
开始处理 219 个文档进行多模态索引...
  已处理 50/219 个原始文档...
  已处理 100/219 个原始文档...
  已处理 150/219 个原始文档...
  已处理 200/219 个原始文档...


Error while downloading from https://cdn-lfs.hf.co/openai/clip-vit-base-patch32/99d28a652e6ec46629ab7047a0ac82c69b1fe11e0ce672c43af65d3a9a3fc05d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1745642738&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NTY0MjczOH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9vcGVuYWkvY2xpcC12aXQtYmFzZS1wYXRjaDMyLzk5ZDI4YTY1MmU2ZWM0NjYyOWFiNzA0N2EwYWM4MmM2OWIxZmUxMWUwY2U2NzJjNDNhZjY1ZDNhOWEzZmMwNWQ%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=DzlFrL5T0qg3SQq3APsz4g56YwU6JbAEugSjSJ3Cy4TJMEUqPro97HjAeFWVaIDSJRc-W12OICCINDJ7mjF6VlxY47yDHL9xiGdf2WlYbsoJ0y7HnuW1vr0AsM9COjQPf5mzBGsXa48DkTlEUWPm5BaBqib83byjvm3RC9PjiyTkt2UZaoX8p3W8xDtH%7EQjV8V3Lq58Ck38PefMi%7EbubT6O8E3C2uzLV5ckXDEZNYrTRFTjRrNoqszXM7pcjz%7EGywDZwre7cNaOCg%7EZioUlpt1SSF5y9u97H%7ENgtPXz2U82zoEujBBiNZeemaNXKkt%7EZ7yx69z54fp61rksAO0mWKw__&Key-Pair-Id=K3RPWS32NS

成功向 Faiss 添加 438 个新向量。当前总数: 438
成功向数据库存储 438 条新记录。
索引过程完成。总处理文档: 219, 跳过重复: 0, 跳过编码错误: 0
索引完成。向量总数: 438, 数据库条目数: 438
--- 步骤 2 完成 ---

--- 步骤 3: 初始化 Retriever (选项 1) ---
Retriever 初始化成功。
--- 步骤 3 完成 ---

--- 步骤 4: 初始化 Generator (选项 1 - MLLM 占位符) ---
Generator (选项 1 - MLLM 占位符) 初始化成功。目标模型: glm-4v, 备用: glm-4-flash
Generator 初始化成功。
--- 步骤 4 完成 ---

--- 步骤 5: 执行查询示例 ---

--- 查询 1/3 ---
查询 (文本): 带隙基准电路的核心原理是什么？
  - 正在 Faiss 索引中搜索 Top 5...
  - Faiss 搜索完成，找到 5 个结果。
  - 正在从数据库获取原始条目信息...
  - 成功获取 5 个条目的数据。
  检索到 5 个条目:
    1. [类型: text] ID: Comparator58, 得分: 0.8904
       文本: 

    ---------------
    2. [类型: text] ID: Bandgap32, 得分: 0.7764
       文本: conclusion": ": 1. **Evidence 1: PTAT Current Generator**\n   - \\( Q1 \\) and \\( Q2 \\): Unequal e...
    ---------------
    3. [类型: text] ID: Bandgap52, 得分: 0.7682
       文本: conclusion": "**\n\nBased on the provided circuit diagram:\n\n1. **Evidence for Bandgap Reference Id...
    ---------------
    4. [类型: text] ID: Bandgap86, 得分: 0.7677
  